In [30]:
import numpy as np
import pandas as pd
import praw
import html
import ast
import re
import ftfy
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer



In [24]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zank7\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zank7\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zank7\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [28]:
# Load Spacy medical model
nlp = spacy.load("en_core_sci_lg")

NameError: name 'spacy' is not defined

In [7]:
user_agent = "AskDoc Scraper"
posts_data = []


In [8]:
rd = praw.Reddit(
    client_id = "_JTrgh_onvOkRlXXuYk7xQ",
    client_secret = "yjEhrCabnwnNb8gxXaQGL17Cn2Nsag",
    user_agent = user_agent
)
subreddit = rd.subreddit('AskDocs')

In [15]:
def scrape_data(sort_type = "hot", limit = 1000):
    try:
        if sort_type not in ['hot', 'top']:
            raise ValueError("sort_type must be 'hot' or 'top'")
        posts_sort = subreddit.top(limit = limit) if sort_type == 'top' else subreddit.hot(limit = limit)
        for index, post in enumerate(posts_sort):
            if index % 10 == 0:
                print(f"Accessed {index} posts") 
                
            post_data = {
                'id': post.id,
                'title': post.title,
                'selftext': post.selftext,
                'author': post.author.name if post.author else None,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': post.created_utc,
                'flair': post.link_flair_text
            }
            
            comments_data = []
            post.comments.replace_more(limit=0)
            for comment in post.comments.list():
                comment_data = {
                    'id': comment.id,
                    'body': comment.body,
                    'author': comment.author.name if comment.author else None,
                    'score': comment.score,
                    'parent_id': comment.parent_id,
                    'created_utc': comment.created_utc,
                    'is_submitter': comment.is_submitter
                }
                comments_data.append(comment_data)
            
            post_data['comments'] = comments_data
            posts_data.append(post_data) 
            
            if index % 100 == 0 and index != 0:  
                initial_df = pd.DataFrame(posts_data)
                initial_df.to_csv(f'reddit_data_{index}.csv', index=False)
                print(f"Data saved at {index} posts")
                
    except Exception as e:
        print(f"An error occurred: {e}")
        initial_df = pd.DataFrame(posts_data)
        initial_df.to_csv('reddit_data_error_save.csv', index=False)
        raise
    
    posts_df = pd.DataFrame(posts_data)
    posts_df.to_csv(f'AskDoc_{sort_type}data.csv', index = False)
    print("Succesfully scraped the data")
    return posts_df


In [16]:
top_posts = scrape_data("top")


Accessed 0 posts
Accessed 10 posts
Accessed 20 posts
Accessed 30 posts
Accessed 40 posts
Accessed 50 posts
Accessed 60 posts
Accessed 70 posts
Accessed 80 posts
Accessed 90 posts
Accessed 100 posts
Data saved at 100 posts
Accessed 110 posts
Accessed 120 posts
Accessed 130 posts
Accessed 140 posts
Accessed 150 posts
Accessed 160 posts
Accessed 170 posts
Accessed 180 posts
Accessed 190 posts
Accessed 200 posts
Data saved at 200 posts
Accessed 210 posts
Accessed 220 posts
Accessed 230 posts
Accessed 240 posts
Accessed 250 posts
Accessed 260 posts
Accessed 270 posts
Accessed 280 posts
Accessed 290 posts
Accessed 300 posts
Data saved at 300 posts
Accessed 310 posts
Accessed 320 posts
Accessed 330 posts
Accessed 340 posts
Accessed 350 posts
Accessed 360 posts
Accessed 370 posts
Accessed 380 posts
Accessed 390 posts
Accessed 400 posts
Data saved at 400 posts
Accessed 410 posts
Accessed 420 posts
Accessed 430 posts
Accessed 440 posts
Accessed 450 posts
Accessed 460 posts
Accessed 470 posts
Acc

In [18]:
hot_posts = scrape_data("hot")

Accessed 0 posts
Accessed 10 posts
Accessed 20 posts
Accessed 30 posts
Accessed 40 posts
Accessed 50 posts
Accessed 60 posts
Accessed 70 posts
Accessed 80 posts
Accessed 90 posts
Accessed 100 posts
Data saved at 100 posts
Accessed 110 posts
Accessed 120 posts
Accessed 130 posts
Accessed 140 posts
Accessed 150 posts
Accessed 160 posts
Accessed 170 posts
Accessed 180 posts
Accessed 190 posts
Accessed 200 posts
Data saved at 200 posts
Accessed 210 posts
Accessed 220 posts
Accessed 230 posts
Accessed 240 posts
Accessed 250 posts
Accessed 260 posts
Accessed 270 posts
Accessed 280 posts
Accessed 290 posts
Accessed 300 posts
Data saved at 300 posts
Accessed 310 posts
Accessed 320 posts
Accessed 330 posts
Accessed 340 posts
Accessed 350 posts
Accessed 360 posts
Accessed 370 posts
Accessed 380 posts
Accessed 390 posts
Accessed 400 posts
Data saved at 400 posts
Accessed 410 posts
Accessed 420 posts
Accessed 430 posts
Accessed 440 posts
Accessed 450 posts
Accessed 460 posts
Accessed 470 posts
Acc

In [8]:
data1 = pd.read_csv('AskDoc_hot_data1.csv')
# data1_copy = data1.copy()
data2 = pd.read_csv('AskDoc_hot_data2.csv')
# data2_copy = data2.copy()
data3 = pd.read_csv('AskDoc_topdata.csv')
# data3_copy = data3.copy()

merged_data = pd.concat([data1, data2, data3], ignore_index = True)
merged_data.reset_index(drop = True, inplace = True)

# print(merged_data)
# print(len(merged_data))

final_data = merged_data.copy()

## Data Preprocessing
Notes for later:

handle null data

handle duplicates

Removing mod comments

Since hot posts may have unmoderated comment, can't just rely on reddit rules to assume the data's good. Thus, gotta use all sorts of methods to clean the posts and comments before fine tuning the llm. My plan:

replace special characters

replace html stuff

replace encoded characters

detect non english language and exclude them

spell check




In [9]:
#null data handling
print(f"Initial number of rows for the scraped data: {len(final_data)}")

Initial number of rows for the scraped data: 6956


In [10]:
mdi = MarkdownIt()

In [13]:
def clean_text(text, author):
    # Skip processing for bot authors or deleted/removed content
    if author in ('AskDocs-ModTeam', 'AutoModerator') or text in ('[removed]', '[deleted]'):
        return None 

    emojis = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    
    # Continue processing if not by a bot or deleted/removed
    text = mdi.render(text)
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = html.unescape(text)
    text = text.lower()  # Normalize case
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = emojis.sub(r'', text)
    
    return text

final_data['title'] = final_data.apply(lambda row: clean_text(row['title'], row['author']), axis=1)
final_data['selftext'] = final_data.apply(lambda row: clean_text(row['selftext'], row['author']), axis=1)


In [14]:
def clean_comments(comments, author):
    comments_list = ast.literal_eval(comments)
    cleaned_comments = []
    for comment in comments_list:
        if comment['author'] not in ('AskDocs-ModTeam', 'AutoModerator') and comment['body'] not in ('[removed]', '[deleted]'):
            cleaned_text = clean_text(comment['body'], comment['author'])
            if cleaned_text is not None:
                cleaned_comments.append(cleaned_text)
    return cleaned_comments

final_data['comments'] = final_data.apply(lambda row: clean_comments(row['comments'], row['author']), axis=1)

In [15]:
new_data = final_data.dropna()
# final_data = final_data[final_data['comments'].str.len() > 5]
print(f"Number of rows after dropping null values: {len(new_data)}")

Number of rows after dropping null values: 4777


In [16]:
new_data.to_csv('CleanData.csv', encoding = 'utf-8', index = False)
# new_data.head(50)

In [20]:
new_data.head(3)

,id,title,selftext,author,score,num_comments,created_utc,flair,comments
1,1cc3km6,mother hospitalized with acute psychosis is i...,i was informed that my mother and best friend ...,thehattedllama,60,33,1.713978e+09,Physician Responded,[sorry this is happening you dont say your mom...
3,1cbjiqd,what are the actual chances of this happening,my son was stillborn last month when i was 37 ...,InstantElla,425,54,1.713915e+09,Physician Responded,[sorry for your loss my deepest sympathies tru...
4,1cbllur,is it weird or unsafe to clean your butthole i...,age 42 sex male height 59 weight 270 race hisp...,tomowudi,228,108,1.713921e+09,Physician Responded,[short answer this is kinda strange but youre ...


In [22]:
def standardize_text(text):
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Load English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Lemmatize tokens and remove stopwords and non-alphabetic tokens
    lemmatized_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words
    ]
    
    # Re-join lemmatized tokens into a single string
    return ' '.join(lemmatized_tokens)


In [25]:
final_data = new_data.copy()
final_data['title'] = final_data['title'].apply(standardize_text)
final_data['selftext'] = final_data['selftext'].apply(standardize_text)
final_data['comments'] = final_data['comments'].apply(lambda x: [standardize_text(comment) for comment in x])


In [26]:
final_data.head()

,id,title,selftext,author,score,num_comments,created_utc,flair,comments
1,1cc3km6,mother hospitalized acute psychosis possible h...,informed mother best friend taken er last nigh...,thehattedllama,60,33,1.713978e+09,Physician Responded,[sorry happening dont say mom age unusual deve...
3,1cbjiqd,actual chance happening,son stillborn last month week morning felt mov...,InstantElla,425,54,1.713915e+09,Physician Responded,[sorry loss deepest sympathy true umbilical co...
4,1cbllur,weird unsafe clean butthole shower finger,age sex male height weight race hispanic durat...,tomowudi,228,108,1.713921e+09,Physician Responded,[short answer kinda strange youre right harmfu...
5,1cc49rs,seeking adviceavenues explore leg paralysis ag...,month ago woke extreme leg pain point couldnt ...,gemmab14,9,3,1.713980e+09,Physician Responded,[usual disclaimer one provide specific medical...
7,1cca3kx,ultrasound report,male lump right testicle complain increased ur...,mangiafazola,5,4,1.713994e+09,Physician Responded,[hello friend im wary firing anything certain ...


In [ ]:
def process_medical_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])


## Data analysis


In [ ]:
def sentiment_analysis(comment):
    return TextBlob(comment).sentiment.polarity

# Apply sentiment analysis on a sample comment
sample_sentiment = sentiment_analysis(final_data['comments'][500][1])  # Analyze the second comment for example
print("Sentiment Polarity:", sample_sentiment)